# Process Plink Results

Read plink results and keep only the test results for the HLA alleles.

In [11]:
import glob
import os
import sys

import pandas as pd

import cdpybio as cpb

In [29]:
outdir = os.path.realpath(os.path.join('../output/process_plink_results'))
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [25]:
fns = glob.glob('/oak/stanford/groups/mrivas/users/jolivier/repos/hla-assoc/data/PLINK_results/*hybrid')
dfs = []
for fn in fns:
    t = cpb.plink.read_logistic2(fn)
    t['code'] = os.path.split(fn)[1].split('.')[0]
    t = t[['code', 'FIRTH?', 'TEST', 'OBS_CT', 'OR', 'SE', 'T_STAT', 'P']]
    dfs.append(t)
results = pd.concat(dfs)

In [30]:
fn = os.path.join(outdir, 'plink_results_all.tsv.gz')
results.to_csv(fn, sep='\t', compression='gzip')

In [32]:
# Filter results by allele and disease frequency